In [199]:
import os
from google.colab import drive
GDRIVE_PATH = '/content/gdrive'
THIS_EXERCISE_PATH = os.path.join(GDRIVE_PATH, "My Drive", "CreditCardGan")
MODELS_HOME = os.path.join(THIS_EXERCISE_PATH, "models")
drive.mount(GDRIVE_PATH)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import os
THIS_EXERCISE_PATH = "."
MODELS_HOME = os.path.join(THIS_EXERCISE_PATH, "models")

In [2]:
import pandas as pd
data = pd.read_csv('https://query.data.world/s/zruvk5e5icrymfmrwrihhzbuaha37h')

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import pickle

In [4]:
with open(os.path.join(MODELS_HOME, 'transformer'), 'rb') as file:
    transformer = pickle.load(file)
data_transformed = torch.tensor(transformer.transform(data), device='cuda', dtype=torch.float)

In [5]:
disc = torch.load(os.path.join(MODELS_HOME, 'discriminator.pt'))

In [6]:
predicted = disc(data_transformed)

In [7]:
predicted = predicted.cpu().detach().numpy()

In [8]:
predicted_aranged = list(zip(predicted.reshape(-1), range(predicted.shape[0])))

In [9]:
predicted_aranged.sort()

# Validation

In [10]:
train, test = train_test_split(range(data_transformed.shape[0]), test_size=0.5, random_state=42)

In [11]:
supposion = list(map(lambda x: x[1], predicted_aranged[len(test):]))

In [12]:
y_true = np.zeros(data_transformed.shape[0])
y_true[train] = 1
y_predicted = np.zeros(data_transformed.shape[0])
y_predicted[supposion] = 1

In [13]:
roc_auc_score(y_true, y_predicted)

0.5009532771253787

In [14]:
data_transformed.shape[0]

284807